In [2]:
import sympy as sp
from sympy import sin, cos, tan, atan, symbols, nsolve, solve, Eq, pi, sqrt
from sympy.physics import units as u
import matplotlib
sp.init_printing()

In [3]:
def rad(degrees):
    return degrees / 180 * pi

## Checking Helical Gear Interference:
Uses formulas 13-19 and 13-25. Solves for maximum gear tooth count.

In [4]:
pinion_tooth_count, k, helix_angle, normal_pa = symbols('N_P k psi phi_n')
phi_t = atan(tan(rad(normal_pa))/cos(rad(helix_angle)))
N_G = (pinion_tooth_count**2*sin(phi_t)**2 - 4*k**2*cos(rad(helix_angle))**2) / (-2*pinion_tooth_count*sin(phi_t)**2 + 4*k*cos(rad(helix_angle)))

N_G.subs({normal_pa: 20,
          helix_angle: 30,
          k: 1, 
          pinion_tooth_count: 16}).evalf()

In [5]:
# Now solving for min helix angle to allow for gear numbers.
pinion, gear = 8, 12 # Target tooth counts.
my_eq = Eq(gear, N_G.subs({normal_pa: 20,
           k: 1, 
           pinion_tooth_count: pinion}))

nsolve(my_eq, helix_angle, 40)

## Approximating diametrial pitch
Uses formula 14-8 and solves for diametrial pitch $P$.

In [6]:

tooth_count = 16
rpm = 310.2
power = 135 * 33000 # convert 135 hp to ft*lbf/min

diametral_pitch, face_width, lewis_ff, stress, velocity_f, t_force = symbols('P F Y sigma K_v W_t') 
pitch_line_velocity = rpm * tooth_count / 2 / diametral_pitch * 2 * pi / 12 

eq_14_8 = Eq(diametral_pitch, face_width * lewis_ff * stress / velocity_f / t_force)

# Values to obtain a lower bound of `P`
conservative_values = {face_width: 3 * pi/diametral_pitch,
                       lewis_ff: 0.24,
                       stress: 22e3, # psi
                       velocity_f: (600+pitch_line_velocity)/600,
                       t_force: power / pitch_line_velocity}

# Values to obtain a upper bound of `P`
nonconservative_values = {face_width: 5 * pi/diametral_pitch,
                       lewis_ff: 0.485,
                       stress: 75e3, # psi
                       velocity_f: sqrt((78 + sqrt(pitch_line_velocity))/78),
                       t_force: power / pitch_line_velocity}

conservative_P = sp.nsolve(eq_14_8.subs(conservative_values), diametral_pitch, 2)
nonconservative_P = sp.nsolve(eq_14_8.subs(nonconservative_values), diametral_pitch, 4)

print(f'Nonconservative estimate of `P` : {nonconservative_P:.3f} teeth/inch')
print(f'Conservative estimate of `P`    : {conservative_P:.3f} teeth/inch')

Nonconservative estimate of `P` : 5.338 teeth/inch
Conservative estimate of `P`    : 1.891 teeth/inch


# Shaft Life Calculations
Solving for min diameter $d$.

Starting with shaft C with gears 4 and 5:

In [ ]:
from sympyplus import *
from sympy import Pow
stage_ratio = 16/38
power = 135 * u.hp
speed_C = 1750 * stage_ratio**2 * u.rpm  # ~ 310.25 rpm

teeth_4 = 38
teeth_5 = 16
dp = 3 / u.inch
normal_pa = rad(20)
helix_angle = rad(30)
trans_pa = atan(tan(normal_pa)/cos(helix_angle)) # ϕ_t ~= 22.80

S = 1 * u.inch
B = 1 * u.inch
F = 2 * u.inch

pitch_velocity_4 = u.convert_to(speed_C * teeth_4/dp/2, [u.ft, u.min])
tangent_load_34 = u.convert_to(power / pitch_velocity_4, [u.lbf])
radial_load_34 = tangent_load_34 * tan(trans_pa)
axial_load_34 = tangent_load_34 * tan(helix_angle)

pitch_velocity_5 = u.convert_to(speed_C * teeth_5/dp/2, [u.ft, u.min])
tangent_load_56 = u.convert_to(power / pitch_velocity_5, [u.lbf])
radial_load_56 = -tangent_load_56 * tan(trans_pa)
axial_load_56 = -tangent_load_56 * tan(helix_angle)

# y direction (radial forces)
# shaft C has three bearings, two gears, split shaft into two portions, left and right
length_C_left = B + 3*S + 2*F
length_C_right = B + 2*S + 1*F
R_ay = -radial_load_34 * (B/2 + S + F/2) / length_C_left
R_cy = -radial_load_56/2
R_by = -radial_load_34 * (B/2 + 2*S + 3/2*F) / length_C_left - radial_load_56/2
# sp.N(R_ay + radial_load_34 + R_by + radial_load_56 + R_cy) 
M_ymax = R_ay * (B/2 + 2*S + 3/2*F) + (R_ay+radial_load_34) * (B/2 + S + F/2) + (R_ay+radial_load_34+R_by) * (length_C_right/2)

# z direction (tangent forces)
R_az = -tangent_load_34 * (B/2 + S + F/2) / length_C_left
R_cz = -tangent_load_56/2
R_bz = -tangent_load_34 * (B/2 + 2*S + 3/2*F) / length_C_left - tangent_load_56/2

sp.N(R_az + tangent_load_34 + R_bz + tangent_load_56 + R_cz)
M_zmax = R_az * (B/2 + 2*S + 3/2*F) + (R_az + tangent_load_34) * (B/2 + S + F/2) + (R_az + tangent_load_34 + R_bz) * (length_C_right/2)

T_C_max = tangent_load_56 * teeth_5 / dp / 2
M_C_max = sqrt(M_ymax**2 + M_zmax**2)

# critical point on shaft C is gear 5 (pinion)
# figure out axial load and direction on bearing (R_b)
bear_thrust_load_max_C = axial_load_34 - axial_load_56

# Endurance limit calcultions
S_ut = 68 # (ksi) start with cheap 1020 cd 
k_a = 2 * S_ut ** (-0.217) # CD
k_b = 0.91 * 2 ** -0.157 # early guess of 2 in, forumula valid 2<d<10
k_c = 1 # loading is not pure torsion
k_d = 1 # high tempature is not a concern
k_e = 0.897 # 90% reliablity
S_ep = min(0.5 * S_ut, 100) # rotary-beam test specimen endurance limit
S_e =  S_ep * k_a * k_b * k_c * k_d * k_e
# DE-Goodman critera for infinite life
# Testing point of gear 5 on shaft c
K_f = 1.74
K_fs = 2.4
T_mC = T_C_max 
M_aC = M_C_max # For a rotating shaft, the constant bending moment will create a completely reversed bending stress
A_ = sqrt(4*(K_f*M_aC)**2) # T_a = 0
B_ = sqrt(3*(K_fs*T_mC)**2) # M_m = 0
n = 1.25

d = Pow(16*n/pi*(A_/S_e/u.ksi + B_/S_ut/u.ksi), Pow(3, -1)) # 7-8 [1]
u.convert_to(d.simplify(), [u.inch]).evalf()
# 3.05 inches

# Try changing material to Ground 4140 Q&T steel
S_ut = 225 
k_a = 1.21 * S_ut ** (-0.067) 
S_ep = min(0.5 * S_ut, 100) 
S_e =  S_ep * k_a * k_b * k_c * k_d * k_e
d = Pow(16*n/pi*(A_/S_e/u.ksi + B_/S_ut/u.ksi), Pow(3, -1)) # 7-8 [1]
u.convert_to(d.simplify(), [u.inch]).evalf()


Now check shaft D with gear 6

In [ ]:
speed_D = 1750 * stage_ratio**3 * u.rpm  # ~ 130.6 rpm
teeth_6 = 38

pitch_velocity_6 = u.convert_to(speed_D * teeth_6/dp/2, [u.ft, u.min])
tangent_load_56 = -u.convert_to(power / pitch_velocity_6, [u.lbf])
radial_load_56 = -tangent_load_56 * tan(trans_pa)
axial_load_56 = -tangent_load_56 * tan(helix_angle)

# y direction (radial forces)
# shaft D has two bearings, one gear, focus on internal part of shaft.
length_D = B + 2*S + F
R_y = -radial_load_56/2 # intermediate load

M_ymax = R_y * length_D/2

# z direction (tangent forces)
R_z = -tangent_load_56/2 # intermediate load

M_zmax = R_z * length_D/2

T_D_max = tangent_load_56 * teeth_6 / dp / 2
M_D_max = sqrt(M_ymax**2 + M_zmax**2)


# DE-Goodman critera for infinite life
# Testing point of gear 6 on shaft D
T_mD = T_D_max 
M_aD = M_D_max
A_ = sqrt(4*(K_f*M_aD)**2) # T_a = 0
B_ = sqrt(3*(K_fs*T_mD)**2) # M_m = 0


d = Pow(16*n/pi*(A_/S_e/u.ksi + B_/S_ut/u.ksi), Pow(3, -1)) # 7-8 [1]
u.convert_to(d.simplify(), [u.inch]).evalf()
# 2.40 inches


## Beam deflection and slope
Use integral relationship to moment: $\theta(x) = \frac{1}{EI}\int{Mdx}$

In [29]:
from sympy import symbols, Piecewise, simplify
E = 30e6 # psi
I = pi * 2.5**4 /64 # in^4

x = symbols('x')
points = x0, x1, x2, x3, x4 = 0, 5.5, 8, 10.5, 13  # Example critical points in inches

# Linear expressions for each span (in*lbf)
M1 = 8072 * (x - x0) / (x1 - x0)
M2 = 8072 * (x2 - x) / (x2 - x1)
M3 = 13944 * (x - x2) / (x3 - x2)
M4 = 13944 * (x4 - x) / (x4 - x3)

# Piecewise function for moment
M_total = Piecewise(
    (M1, (x >= x0) & (x < x1)),
    (M2, (x >= x1) & (x < x2)),
    (M3, (x >= x2) & (x < x3)),
    (M4, (x >= x3) & (x <= x4))
)

M_total_simplified = simplify(M_total)
theta = M_total_simplified.integrate(x) / E / I
y = theta.integrate(x)

# plot(y, theta)

points = (0.5, *points)
for x_val in points:
    print(f'{x_val} in : {theta.subs({x: x_val}).evalf(2)} & {y.subs({x:x_val}).evalf(2)}')

0.5 in : 0.0000032 & 5.3E-7
0 in : nan & nan
5.5 in : 0.00039 & 0.00071
8 in : 0.00056 & 0.0020
10.5 in : 0.00086 & 0.0036
13 in : 0.0012 & 0.0063
